In [1]:
!pip install -q transformers sentence-transformers faiss-cpu pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [2]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]


Saving Intern Task GenAI.pdf to Intern Task GenAI.pdf


In [3]:
import pdfplumber

def extract_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                text += page.extract_text() + "\n"
    return text.strip()

text = extract_text(pdf_path)
print(text[:1000])  # Preview


TASK: Smart Assistant for Research Summarization
DEADLINE :
Submit within 7 days from the date of assignment.
Objective:
EZ has designed this task to evaluate candidates on their ability to build
AI-powered tools that go beyond basic automation and demonstrate contextual
understanding and reasoning.
The objective is to develop an AI assistant that not only reads content from
documents but can also understand and reason through it.
You are required to create a document-aware assistant capable of handling both
free-form question answering and logic-based question generation using
user-uploaded documents (in PDF or TXT format).
Problem Statement:
Reading through large documents like research papers, legal files, or technical
manuals is time-consuming. Traditional summarizers or keyword search tools fall
short when it comes to deep comprehension and logical reasoning.
Build a GenAI assistant that reads user-uploaded documents and can:
● Answer questions that require comprehension and infer

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(text, chunk_size=500, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)

chunks = chunk_text(text)


In [5]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings
embeddings = model.encode(chunks)

# Build FAISS vector index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

def retrieve_top_chunks(query, k=3):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, k)
    return [chunks[i] for i in I[0]]

def ask_question(query):
    context = " ".join(retrieve_top_chunks(query))
    result = qa_model(question=query, context=context)
    print("Q:", query)
    print("A:", result['answer'])

# Test it
ask_question("What is the document about?")


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


Q: What is the document about?
A: a structured English report, research paper


In [8]:
ask_question("how many days are given to submit the assignment")

Q: how many days are given to submit the assignment
A: 7
Q: how many days are given to submit the assignment
A: 7


In [9]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize(text, max_words=150):
    short_text = text[:3000]
    result = summarizer(short_text, max_length=150, min_length=30, do_sample=False)
    return result[0]['summary_text']

print("📄 Summary:\n", summarize(text))


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


📄 Summary:
 EZ has designed this task to evaluate candidates on their ability to build tools that go beyond basic automation and demonstrate contextual understanding and reasoning. The objective is to develop an AI assistant that not only reads content from but can also understand and reason through it.


In [1]:
!pip install -q gradio pdfplumber sentence-transformers faiss-cpu transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [2]:
import gradio as gr
import pdfplumber
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import numpy as np

# Models
embedder = SentenceTransformer("all-MiniLM-L6-v2")
qa_model = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Global state
chunks = []
index = None

# 🧾 Step 1: Extract text
def extract_text(pdf_file):
    with pdfplumber.open(pdf_file.name) as pdf:
        text = ""
        for page in pdf.pages:
            if page.extract_text():
                text += page.extract_text() + "\n"
    return text.strip()

# 🔍 Step 2: Chunk text
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
    return chunks

# 🧠 Step 3: Create vector store
def build_index(chunks):
    vectors = embedder.encode(chunks)
    dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(vectors))
    return index

# 🔄 Step 4: Full handler on upload
def handle_upload(pdf_file):
    global chunks, index
    text = extract_text(pdf_file)
    chunks = chunk_text(text)
    index = build_index(chunks)
    sample_text = " ".join(chunks[:3])[:3000]
    summary = summarizer(sample_text, max_length=150, min_length=30, do_sample=False)[0]["summary_text"]
    return summary


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [3]:
# ❓ Ask Anything
def ask_question(question):
    global chunks, index
    if not chunks or not index:
        return "Please upload a PDF first."
    query_vector = embedder.encode([question])
    D, I = index.search(query_vector, k=3)
    context = " ".join([chunks[i] for i in I[0]])
    result = qa_model(question=question, context=context)
    return result["answer"]

# 🎯 Challenge Me
def challenge_me():
    global chunks
    if not chunks:
        return "Please upload a PDF first."

    sample_text = " ".join(chunks[:3])[:3000]
    logic_prompt = f"""
From the text below, generate 3 logic-based or inference questions with their correct answers.

Text:
{sample_text}

Format:
Q1: ...
A1: ...
Q2: ...
A2: ...
Q3: ...
A3: ...
"""
    question_gen = pipeline("text-generation", model="gpt2")
    result = question_gen(logic_prompt, max_new_tokens=300, do_sample=False)[0]["generated_text"]
    return result


In [4]:
with gr.Blocks() as demo:
    gr.Markdown("## 📄 GenAI Research Assistant with Hugging Face (Colab UI)")

    with gr.Row():
        pdf_input = gr.File(label="Upload PDF")
        summary_output = gr.Textbox(label="🧠 Auto Summary", lines=4)

    with gr.Row():
        question_input = gr.Textbox(label="Ask Anything")
        question_output = gr.Textbox(label="Answer")

    with gr.Row():
        challenge_output = gr.Textbox(label="🧪 Challenge Me Questions", lines=10)

    upload_btn = gr.Button("📤 Upload & Summarize")
    ask_btn = gr.Button("💬 Ask")
    challenge_btn = gr.Button("🎯 Challenge Me")

    upload_btn.click(fn=handle_upload, inputs=pdf_input, outputs=summary_output)
    ask_btn.click(fn=ask_question, inputs=question_input, outputs=question_output)
    challenge_btn.click(fn=challenge_me, outputs=challenge_output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://45920ef89e81474b25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
